In [ ]:
#!pip uninstall -y langchain langchain_core langchain_community langchain_openai
#!pip install "langchain_community==0.2.5"
!pip install --upgrade langchain langchain_core "langchain_community==0.2.5" langchain_openai
#!pip install pypdf
!pip install markdown weasyprint
!pip install -q pymupdf python-docx deepeval langchain langchain-openai
#!apt-get install -y libpango-1.0-0 libcairo2
!pip install markdown-it-py
!pip install pymupdf
!pip install cairosvg
!pip install deepeval

!git clone https://github.com/rilianx/python_libs.git

  Using cached langchain-1.2.6-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_openai-1.1.7-py3-none-any.whl.metadata (2.6 kB)
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_openai-1.1.6-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_openai-1.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_openai-1.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_openai-1.1.3-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_openai-1.1.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_openai-1.1.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchain_openai-1.1.0-py3-none-any.whl.metadata (2.6 kB)
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_openai-1.0.3-

In [ ]:
# --- FASE 0: INFRAESTRUCTURA ---
import os
import re
from langchain_openai import ChatOpenAI
from deepeval import evaluate
from deepeval.metrics import FaithfulnessMetric, GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.models.base_model import DeepEvalBaseLLM
from google.colab import userdata

from google.colab import drive
drive.mount('/content/drive')

!ln -s "/content/drive/MyDrive/20252_Informes" "/content/20252_Informes"
base_folder = "/content/20252_Informes/Muestra_Informe2"

# Configura API
os.environ["OPENAI_API_KEY"] = userdata.get('APIGPT')

print("✅ Fase 0: Librerías cargadas correctamente.")

Mounted at /content/drive
✅ Fase 0: Librerías cargadas correctamente.


In [ ]:
# --- FASE 0.5: ENRIQUECIMIENTO DE IMÁGENES ---
# Se descomprimen los .zips y crea "enriched_mds" procesando las imagenes y
# añadiendo las descripciones de las imagenes, codigos y graficos


import os
import glob
from pathlib import Path
from openai import OpenAI
from python_libs.enrich_md import extrae_imagenes_base64, unzip_markdown, extract_image_descriptions, enrich_markdown

# 1. Configuración Inicial
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
folder_path = base_folder  # Usa la variable definida en la celda anterior

# 2. CONFIGURACIÓN BDD: Reglas de descripción de elementos

BDD_IMAGENES = {
    "codigo": """
    - TIPO: Captura de Código / Terminal.
    - REGLA: Transcribe el código visible. Si hay errores resaltados, menciónalos.
    - EJEMPLO: 'Bloque de código Python. Error de indentación en línea 4.'
    """,
    "diagramas": """
    - TIPO: Diagramas UML / Flujo.
    - REGLA: Describe el flujo lógico (origen -> destino).
    - EJEMPLO: 'Diagrama de secuencia. Usuario -> API -> Base de Datos.'
    """,
    "graficos": """
    - TIPO: Gráficos Estadísticos.
    - REGLA: Menciona ejes, tendencias y máximos/mínimos.
    - EJEMPLO: 'Gráfico de barras. Eje X: Tiempo, Y: Costo. Tendencia lineal.'
    """
}

# 3. PROMPT MAESTRO

describe_image_prompt_template = f"""
Eres un experto en accesibilidad técnica. Describe esta imagen para un estudiante ciego.
PASO 1: CLASIFICACIÓN (Identifica si es {', '.join(BDD_IMAGENES.keys())}).
PASO 2: APLICACIÓN DE ESTÁNDARES:
SI ES CÓDIGO: {BDD_IMAGENES['codigo']}
SI ES DIAGRAMA: {BDD_IMAGENES['diagramas']}
SI ES GRÁFICO: {BDD_IMAGENES['graficos']}
PASO 3: SALIDA. Genera solo la descripción directa.
"""

# 4. EJECUCIÓN DEL FLUJO
# aca me pierdo xd
paths = glob.glob(os.path.join(folder_path, "*.zip"))
print(f"📂 Encontrados {len(paths)} archivos .zip para procesar.")

for path in paths:
    print(f"\n📄 Procesando: {os.path.basename(path)}")

    # Chequeo si ya existe
    nombre_base = os.path.basename(path).split('_')[0]
    if os.path.exists(f"{base_folder}/enriched_mds/{nombre_base}.md"):
        print("   ⏭️  Ya existe, saltando.")
        continue

    try:
        # A. Descomprimir
        temp_dir = unzip_markdown(zip_path=path, dest_dir=f"{base_folder}/{nombre_base}")

        # B. Extraer imágenes
        md_path = glob.glob(os.path.join(temp_dir, '**', '*.md'), recursive=True)[0]
        extrae_imagenes_base64(md_path)

        # C. Generar descripciones con IA (Usando el Prompt Maestro)
        print("   👁️  Analizando imágenes con IA...")
        md_text, image_descriptions = extract_image_descriptions(
            client,
            temp_dir,
            describe_image_prompt_template
        )

        # D. Auditoría de Calidad segun cantidad de letras
        imgs_vagas = 0
        for img_name, desc in image_descriptions.items():
            if len(desc) < 40 or ("imagen muestra" in desc.lower() and len(desc) < 60):
                print(f"      ⚠️  Alerta: Descripción vaga en '{img_name}'")
                imgs_vagas += 1

        if imgs_vagas == 0 and image_descriptions:
            print(f"      ✅ Calidad OK ({len(image_descriptions)} imágenes)")

        # E. Guardar Resultado
        contenido = enrich_markdown(md_text, image_descriptions)

        zip_path_obj = Path(path.strip())
        dest_dir = zip_path_obj.parent / "enriched_mds"
        dest_dir.mkdir(parents=True, exist_ok=True)
        dest_path = dest_dir / (nombre_base + ".md")

        with open(dest_path, "w") as f:
            f.write(contenido)
        print(f"   💾 Guardado en: enriched_mds/{dest_path.name}")

    except Exception as e:
        print(f"   ❌ Error crítico en {path}: {e}")
        continue

📂 Encontrados 14 archivos .zip para procesar.

📄 Procesando: Equestria Bytes_137208_assignsubmission_file_Informe 2 - Equestria Bytes.zip
   ⏭️  Ya existe, saltando.

📄 Procesando: Chinche y Bolinche_137190_assignsubmission_file_Informe 2 - Chinche y Bolinche.zip
   ⏭️  Ya existe, saltando.

📄 Procesando: Ensalada Sin Cesar_137321_assignsubmission_file_Easy problem.zip
   ⏭️  Ya existe, saltando.

📄 Procesando: Algorítmicos Anónimos_137180_assignsubmission_file_informe.zip
   ⏭️  Ya existe, saltando.

📄 Procesando: El terror de los PCs_137207_assignsubmission_file_The Least Round Way (Reporte 2) El terror de los PCs.zip
   ⏭️  Ya existe, saltando.

📄 Procesando: c_137965_assignsubmission_file_effe3030-64f3-44ac-bf7c-6129ecccefaf_ExportBlock-d819a996-6c43-4049-a61f-715ad1ea843f.zip
   ⏭️  Ya existe, saltando.

📄 Procesando: ADAmantium_137186_assignsubmission_file_Informe 2 ICI2242 ADAmantium.zip
   ⏭️  Ya existe, saltando.

📄 Procesando: Ctrl+Z_137191_assignsubmission_file_INFORME II 

In [ ]:
# --- FASE 1: PRE-PROCESAMIENTO Y ANONIMIZACIÓN Y LECTURA ---

def leer_pdf(ruta_archivo):

    """ Recibe ruta con archivo ".pdf"
        Retorna texto

        Extrae texto de PDF usando PyMuPDF (fitz)"""
    texto_completo = ""
    try:
        doc = fitz.open(ruta_archivo)
        for pagina in doc:
            texto_completo += pagina.get_text() + "\n"
        return texto_completo
    except Exception as e:
        return f"Error leyendo PDF: {e}"

def leer_docx(ruta_archivo):
    """
    Recibe ruta con archivo ".docx"
    Retorna texto

    Extrae texto de Word usando python-docx"""
    texto_completo = ""
    try:
        doc = Document(ruta_archivo)
        for para in doc.paragraphs:
            texto_completo += para.text + "\n"
        return texto_completo
    except Exception as e:
        return f"Error leyendo DOCX: {e}"

def leer_archivo_inteligente(ruta_archivo):
    """
    Recibe ruta con archivo ".pdf", ".docx" o ".md"
    Retorna texto

    Detector de formato: Decide qué función usar según la extensión.
    """
    extension = os.path.splitext(ruta_archivo)[1].lower()

    if extension == '.pdf':
        return leer_pdf(ruta_archivo)
    elif extension == '.docx':
        return leer_docx(ruta_archivo)
    elif extension in ['.md', '.txt']:
        # Lectura directa para Markdown o Texto plano
        with open(ruta_archivo, 'r', encoding='utf-8', errors='ignore') as f:
            return f.read()
    else:
        return "Formato no soportado. Usa PDF, DOCX o MD."

import re
def anonimizar_texto(texto):
    """
    Recibe texto
    Retorna texto anonimizado y mapa de seguridad
    """
    mapa_seguridad = {}
    texto_anonimo = texto
    # 1. Emails
    emails = re.findall(r'[\w\.-]+@[\w\.-]+\.\w+', texto)
    for i, email in enumerate(set(emails)):
        token = f"[EMAIL_{i}]"
        mapa_seguridad[token] = email
        texto_anonimo = texto_anonimo.replace(email, token)

    return texto_anonimo, mapa_seguridad

def desanonimizar_texto(texto_generado, mapa_seguridad):
    """
    Recibe texto anonimizado y mapa de seguridad
    Retorna texto original
    """
    texto_final = texto_generado
    for token, valor_real in mapa_seguridad.items():
        texto_final = texto_final.replace(token, valor_real)
    return texto_final

print("✅ Fase 1: Conversión de archivos (PDF/DOCX/MD) lista.")

✅ Fase 1: Conversión de archivos (PDF/DOCX/MD) lista.


In [ ]:
# --- FASE 2: CONTEXTO ---

# Esto simula base de datos de "Buenos y Malos ejemplos"

#  DEFINICIÓN DE PROMPTS MODULARES ---


#Asignaciones de roles segun seccion.
#Asignaciones de intrucciones segun seccion
#Intrucciones de evaluacion y salida

PROMPTS_MODULARES= {
# ============================================================================
#                             SECCIÓN: FORMA
# ============================================================================
    "forma": """
Rol: Eres un Auditor de Calidad de Documentación Técnica. Tu misión es validar que la información necesaria esté presente, independientemente del formato exacto.

CONTEXTO: Estás evaluando informes de estudiantes de Algoritmos. A menudo usan terminología variada.

Instrucciones de Búsqueda Flexible (Anti-Alucinación):
1.  **Sinónimos:** Si buscas "Entrada", acepta también: "Input", "Datos", "Parámetros", "Consideraciones iniciales" o simplemente una lista de variables al inicio.
2.  **Variables:** No busques letras específicas ($N, M$). Busca la *definición* (ej: "un entero que representa el tamaño").
3.  **Gráficos:** Si el eje dice "Segundos", "s", "t(s)" o "Tiempo", es válido. No exijas un estándar ISO rígido.

Evalúa los siguientes puntos:
- Definición de Entradas/Salidas: ¿Se describe qué recibe el algoritmo y qué devuelve? (Busca descripciones semánticas, no solo tipos de datos).
- Ejemplos Concretos: ¿Hay algún caso de prueba, ejemplo juguete o traza con valores reales? (Busca en texto o bloques de código).
- Uso de Imágenes/Diagramas: ¿Hay apoyo visual? (Si la imagen se entiende por sí misma o tiene título, es válido aunque no tenga descripción larga).
- Calidad de Gráficos: ¿Los ejes tienen etiquetas que indiquen qué se mide?
- Selección de Gráfico: ¿Es el gráfico adecuado para los datos mostrados?
- Formato y Ortografía: Calidad general del documento.

Para cada punto, genera:
1. **Evidencia Encontrada**: Cita el texto o describe dónde está. SI NO LO ENCUENTRAS, escribe: "No se halló evidencia explícita ni implícita".
2. **Análisis**: ¿Es claro? ¿Sirve a su propósito?
3. **Calificación**: [No cumple | Cumple parcialmente |Bien hecho]
4. **Retroalimentación**: (Usa un tono constructivo. Si el elemento está pero es confuso, di cómo ordenarlo. No digas que "no existe" si está desordenado).

Genera la sección: "## Análisis de Forma y Estructura".
""",
#======================================================================================
#                              SECCIÓN: RIGUROSIDAD
#======================================================================================
    "rigurosidad": """
Rol: Eres un Profesor Senior de Algoritmos. Valoras el razonamiento lógico por encima de la memorización de nombres.

Instrucciones Críticas de Evaluación Lógica:
1.  **Concepto > Nombre:** En "Subestructura Óptima", el alumno debe demostrar que "una solución óptima global se compone de soluciones óptimas locales". Si explica esto correctamente PERO no menciona "Bellman", califica como APROBADO (Cumple Satisfactoriamente). Solo exige "Bellman" para "Sobresaliente".
2.  **Validación de Código:** Al evaluar "Correspondencia Algorítmica", busca en el código patrones de memoización como: diccionarios, matrices, condiciones `if x in memo`, o decoradores `@lru_cache`.
3.  **Complejidad:** Acepta derivaciones lógicas. Si el alumno dice "Llenar una tabla de tamaño N*M cuesta O(1) por celda -> O(NM)", es válido. No exijas una fórmula rígida si el razonamiento es correcto.

Evalúa los siguientes puntos:
- Justificación de la Subestructura Óptima: ¿Argumenta la independencia de los subproblemas? (Ver instrucción 1).
- Formalización de la Recurrencia: ¿Define matemáticamente $F(estado)$ y sus Casos Base?
- Derivación de Complejidad: ¿Relaciona el tamaño de la estructura (estados) con el costo de calcular cada estado?
- Correspondencia Algorítmica: ¿El código implementa la lógica recursiva/iterativa descrita en la teoría?
- Análisis Crítico: ¿Explica las causas del rendimiento observado en los gráficos?
- Validación (Dry Run): ¿Existe una traza manual o explicación paso a paso de un caso pequeño?
- Limitaciones: ¿Menciona problemas de memoria, pila de recursión o casos borde?

Formato de Salida por punto:
1. **Evidencia**: Cita la justificación del alumno.
2. **Análisis de Profundidad**: Evalúa si entiende el "por qué" o solo describe el "cómo".
3. **Calificación**: [No cumple | Cumple parcialmente | Bien hecho]
4. **Retroalimentación**: Sé específico. Si falta profundidad, da una pista pedagógica (ej: "¿Qué pasaría si la decisión local no fuera óptima?").

Genera la sección: "## Análisis de Rigurosidad Técnica".
""",

    # SECCIÓN: CONSISTENCIA
    "consistencia": """
Rol: Eres un Auditor Lógico. Buscas contradicciones fácticas.

Instrucciones de Cross-Check (Verificación Cruzada):
1.  **Lectura Comprensiva:** Antes de marcar una contradicción, verifica si el alumno
usa notación equivalente (ej: Texto dice $O(N^2)$, Gráfico es una parábola
-> ES CONSISTENTE).
2.  **Código vs. Teoría:** Verifica si la fórmula matemática se traduce al código.
 (Ej: Si la fórmula tiene `min(...)`, el código debe tener `min(...)` o lógica equivalente).

Evalúa:
- Teoría vs. Código: Coincidencia entre fórmula y script.
- Complejidad vs. Gráficos: ¿La curva visual respalda la Big-O declarada?
- Notación Unificada: ¿Las variables significan lo mismo en todo el doc?
- Anti-Relleno: ¿Hay redundancia excesiva?
- Respaldo de Datos: ¿Las conclusiones se basan en los experimentos mostrados?

Formato de Salida por punto:
1. **Evidencia Comparativa**: "Texto dice X, Gráfico muestra Y".
2. **Veredicto**: ¿Coherente o Contradictorio?
3. **Calificación**: [No cumple | Cumple parcialmente | Bien hecho]
4. **Retroalimentación**: Señala la discrepancia exacta si existe.

Genera la sección: "## Análisis de Consistencia y Coherencia".
"""
}


# Prompt de calificacion de analisis
# Criterios  No cumple | Cumple Parcialmente | Bien hecho

PROMPT_RESUMEN_FINAL = """
ROL: Eres un motor de procesamiento de datos backend. Tu función NO es conversar, sino estructurar datos crudos en una plantilla Markdown fija.

TAREA DE PROCESAMIENTO:
1.  Lee los análisis parciales proporcionados (Forma, Rigurosidad, Consistencia).
2.  **CONTABILIZA** manualmente cuántas veces aparece cada calificación ("Bien hecho", "Cumple parcialmente", "No cumple") dentro de los puntos de cada sección.
3.  **CALCULA** la "Calificación promedio" de la sección basándote en la mayoría (moda).
4.  **RELLENA** la plantilla de abajo reemplazando los marcadores entre llaves `{...}` con los datos reales.

--- PLANTILLA DE SALIDA OBLIGATORIA (COPIAR Y RELLENAR) ---

# Evaluación Final y Resumen

## 1. Tabla Resumen de Calificaciones

| Sección | Calificación promedio | Desglose (Bien hecho / Cumple parcialmente / No cumple) |
| :--- | :---: | :---: |
| Forma y Estructura | {CALIFICACION_GLOBAL_FORMA} | {CANT_BIEN}/{CANT_PARCIAL}/{CANT_NO} |
| Rigurosidad Técnica | {CALIFICACION_GLOBAL_RIGUROSIDAD} | {CANT_BIEN}/{CANT_PARCIAL}/{CANT_NO} |
| Consistencia y Coherencia | {CALIFICACION_GLOBAL_CONSISTENCIA} | {CANT_BIEN}/{CANT_PARCIAL}/{CANT_NO} |

## 2. Conclusión General
{PARRAFO_DE_CONCLUSION_GLOBAL}

## 3. Puntos Críticos de Mejora
1. **{TITULO_CORTO_PUNTO_1}**: {EXPLICACION_DEL_PORQUE_1}
2. **{TITULO_CORTO_PUNTO_2}**: {EXPLICACION_DEL_PORQUE_2}
3. **{TITULO_CORTO_PUNTO_3}**: {EXPLICACION_DEL_PORQUE_3}

--- REGLAS DE SILENCIO (STRICT MODE) ---
1.  **PROHIBIDO HABLAR**: No incluyas saludos ("Hola", "Aquí está el reporte"), introducciones ("Basado en el análisis...") ni despedidas.
2.  **INICIO EXACTO**: Tu respuesta DEBE comenzar con el carácter "#" de la primera línea de la plantilla.
3.  **FORMATO NUMÉRICO**: En la columna "Desglose", usa estrictamente el formato "N/N/N" (ejemplo: 2/4/0). No uses palabras.
"""

print("Fase 2: Prompts Modulares cargados.")

#FEW SHOT POR DIMENSION

BDD_EJEMPLOS_POR_DIMENSION = {
# ------------------------------------------------------------------
#                                       SECCIÓN: FORMA
# Objetivo: Enseñar al modelo a ser flexible con la ubicación pero estricto con la claridad.
# ------------------------------------------------------------------
    "forma": {
        "buena_practica": """
        [Ejemplo Positivo 1 - Sugerencia Constructiva]
        'Aunque defines las entradas en el código, la sección "Descripción del Problema" no las menciona.
        Sugerencia: Mueve la lista de variables ($N, K, S$) del código al texto principal para mejorar la legibilidad.'

        [Ejemplo Positivo 2 - Calidad de Gráficos]
        'El gráfico de tiempos es legible, pero el eje Y no tiene unidades. No sabemos si son milisegundos o segundos.
        Por favor, agrega la etiqueta "Tiempo (ms)" en la próxima entrega.'
        """,

        "mala_practica": """
        [A EVITAR - Vago]
        'El formato está feo y desordenado.' (No dice qué arreglar).

        [A EVITAR - Falso Negativo por rigidez]
        'No cumple: No encontré la sección titulada "Entradas".'
        (Error: El alumno las puso bajo el título "Datos", el auditor debe ser capaz de encontrarlo).
        """
    },

    # ------------------------------------------------------------------
    #                            SECCIÓN: RIGUROSIDAD
    # Objetivo: Enseñar al modelo a evaluar la LÓGICA, no solo el vocabulario.
    # ------------------------------------------------------------------
    "rigurosidad": {
        "buena_practica": """
        [Ejemplo Positivo 1 - Profundidad Lógica]
        'Explicas correctamente cómo divides el problema, pero te falta justificar la independencia.
        ¿Por qué estás seguro de que elegir el óptimo local en el paso $i$ no arruina la solución global en el paso $i+1$?
        Agrega un argumento lógico o una prueba por contradicción.'

        [Ejemplo Positivo 2 - Complejidad]
        'Dices que es $O(N)$, pero en la línea 45 de tu código hay un slicing de string `s[i:j]` dentro de un bucle.
        Esto añade un factor oculto de costo $O(N)$. La complejidad real parece ser $O(N^2)$. Corrígelo o justifica por qué es O(1).'
        """,

        "mala_practica": """
        [A EVITAR - Pedantería]
        'No mencionas explícitamente el "Principio de Bellman".'
        (Mal feedback: Si el alumno explicó la idea correctamente, no importa si olvidó
        el nombre del matemático).

        [A EVITAR - Superficialidad]
        'El código se ve bien.'
        (Debe validar si la lógica recursiva tiene casos base y memoización, no solo si "se ve bien").
        """
    },

    # ======================================================================================
    #                         SECCIÓN: CONSISTENCIA
    #   Objetivo : Evitar alucinaciones y que haya coherencia en el texto
    # ======================================================================================
    "consistencia": {
        "buena_practica": """
        [Ejemplo Positivo 1 - Alucinación Texto/Gráfico]
        'Hay una contradicción: En el texto afirmas que el algoritmo es lineal $O(N)$, pero la Figura 3 muestra claramente
        una curva parabólica (crecimiento cuadrático). Revisa tu análisis teórico o tus datos experimentales.'

        [Ejemplo Positivo 2 - Discrepancia Teoría/Código]
        'En la fórmula matemática defines $F(i)$, pero en el código tu función recibe dos parámetros `def solucion(i, j)`.
        Actualiza la fórmula en LaTeX para que coincida con la implementación real.'
        """,

        "mala_practica": """
        [A EVITAR - Generalización]
        'El informe es inconsistente.' (Debes señalar EXACTAMENTE dónde chocan los datos).
        [A EVITAR - Falso Positivo en Gráficos]
        'El gráfico no tiene unidades.'
        (Error: Si el gráfico dice "Tiempo (s)", la 's' es la unidad. No penalices si usa abreviaciones estándar).

        [A EVITAR - Inferencia Falsa]
        'Dices que usas Python pero el código parece C++.'
        (A menos que sea obvio, no adivines; cíñete a errores lógicos claros).
        """
    }
}

#Se generan prompts enriquecidos, se utiliza la base y la seccion, se le agrega tambien
# "few-shot" de las buenas y malas practicas

def obtener_prompt_enriquecido(rubrica_base, dimension):
    """
    Inyecta ejemplos específicos según la dimensión que se esté evaluando.
    """
    # Obtenemos los ejemplos específicos o usamos unos genéricos por seguridad

    datos_bdd = BDD_EJEMPLOS_POR_DIMENSION.get(dimension, {
        "buena_practica": "Sé específico y constructivo.",
        "mala_practica": "No seas vago ni agresivo."
    })

    prompt = f"""
    {rubrica_base}

    --- REGLAS DE FORMATO ESTRICTAS ---
    1.  NO incluyas saludos (ej: "Hola", "Aquí está el análisis").
    2. NO incluyas despedidas ni ofrecimientos de ayuda futura (ej: "Espero que sirva", "¿Quieres que analice otro?").
    3. NO des consejos sobre qué hacer a continuación fuera del feedback solicitado.
    4. COMIENZA tu respuesta INMEDIATAMENTE con el primer título Markdown (# Informe...).
    5. Tu salida debe ser exclusivamente el contenido técnico solicitado.


    --- GUÍA DE ESTILO ESPECÍFICA PARA: {dimension.upper()} ---

    LO QUE BUSCAMOS (Buena Práctica):
    {datos_bdd['buena_practica']}

    LO QUE DEBES EVITAR (Mala Práctica):
    {datos_bdd['mala_practica']}

    Usa este estilo para generar tu crítica.
    """
    return prompt


print("✅ Fase 2: Conexión a BDD .")

Fase 2: Prompts Modulares cargados.
✅ Fase 2: Conexión a BDD .


In [ ]:
# --- FASE 3 NÚCLEO DE GENERACIÓN Y AUDITORÍA GRANULAR ---

#from deepeval.test_case import LLMTestCase, LLMTestCaseParams AAAAA

# Definimos el modelo
NOMBRE_MODELO = "gpt-5-mini"
llm_engine = ChatOpenAI(model=NOMBRE_MODELO, temperature=1) #Para usar el 5-mini tuve que poner temperature=1, para el 4o-mini era 0

# Wrappe
class ModeloPersonalizado(): #Dentro tenía DeepEvalBaseLLM
    def __init__(self, model_name=NOMBRE_MODELO):
        self.model_name = model_name
        self.model = llm_engine
    def load_model(self): return self.model
    def generate(self, prompt): return self.model.invoke(prompt).content
    async def a_generate(self, prompt): res = await self.model.ainvoke(prompt); return res.content
    def get_model_name(self): return self.model_name

mi_modelo_eval = ModeloPersonalizado()

#Base para generar analisis

def generar_y_auditar_modular(texto_anonimo):
    feedback_acumulado = "# Informe de Retroalimentación Automática\n\n"
    scores_totales = []
    detalles_auditoria = {}

    print(f"🤖 Iniciando análisis modular con {NOMBRE_MODELO}...")

    # 1. Bucle paso a paso (Análisis por dimensiones)
    for dimension, prompt_instruccion in PROMPTS_MODULARES.items():
        print(f"\n🔹 Procesando dimensión: {dimension.upper()}")

        prompt_final = obtener_prompt_enriquecido(prompt_instruccion, dimension)

        mensaje = f"""
        {prompt_final}

        --- INFORME A EVALUAR ---\n{texto_anonimo}\n
        --- INSTRUCCIÓN ---\nGenera el análisis solicitado con su calificación.
        """

        # Invocación al LLM
        respuesta_parcial = llm_engine.invoke(mensaje).content

        # Auditoría (Se mantiene igual) AAAA
        #score, razon = auditar_seccion(texto_anonimo, respuesta_parcial, prompt_instruccion, dimension)
        #scores_totales.append(score)
        #detalles_auditoria[dimension] = {"score": score, "reason": razon}

        feedback_acumulado += respuesta_parcial + "\n\n---\n\n"
        #print(f"   ✅ Score {dimension}: {score:.2f} | Feedback generado.") AAAA

    # 2.Generación del Resumen Final Integrador
    print(f"\n🔹 Generando Evaluación Final y Resumen...")

    mensaje_resumen = f"""
    {PROMPT_RESUMEN_FINAL}

    --- ANÁLISIS DETALLADOS PREVIOS ---
    {feedback_acumulado}

    --- INSTRUCCIÓN ---
    Genera el resumen final basándote únicamente en los análisis previos.
    """

    respuesta_final = llm_engine.invoke(mensaje_resumen).content
    feedback_acumulado += respuesta_final # Añadimos el resumen al final del documento

    # Cálculo de score global
    #score_global = sum(scores_totales) / len(scores_totales) if scores_totales else 0 AAAA
    score_global = 1
    return feedback_acumulado, score_global, detalles_auditoria

print("✅ Fase 3 Actualizada: Motor con Resumen Final listo.")

✅ Fase 3 Actualizada: Motor con Resumen Final listo.


In [ ]:
# --- FASE 4: PROCESAMIENTO FINAL ---

def procesar_flujo_modular(ruta_archivo):
    print(f"📄 Procesando: {ruta_archivo}")

    # 1. Leer y Anonimizar
    contenido_raw = leer_archivo_inteligente(ruta_archivo)
    contenido_seguro, mapa_ids = anonimizar_texto(contenido_raw)

    # 2. Flujo Integrado (Genera y Audita por partes)
    feedback_completo, score_global, detalles = generar_y_auditar_modular(contenido_seguro)

    # 3. Vista previa

    print("\n" + "="*40)
    print("VISTA PREVIA DEL FEEDBACK:")
    print("="*40)
    print(feedback_completo)
    print("="*40)

    # 4. Guardar
    feedback_real = desanonimizar_texto(feedback_completo, mapa_ids)
    nombre_salida = f"Feedback_Final_{os.path.basename(ruta_archivo)}"

    with open(nombre_salida, "w") as f:
        f.write(feedback_real)

    print(f"\n✅ ¡Listo! Archivo guardado como: {nombre_salida}")



In [ ]:
# - AUMENTO DE TIMEOUT ---
import os

os.environ["DEEPEVAL_PER_TASK_TIMEOUT_SECONDS_OVERRIDE"] = "600"
os.environ["DEEPEVAL_TASK_GATHER_BUFFER_SECONDS_OVERRIDE"] = "600"

print("✅ Configuración de Timeout actualizada a 10 minutos.")

✅ Configuración de Timeout actualizada a 10 minutos.


In [ ]:
# --- EJECUCIÓN DE PRUEBA ---
informe_md = f"{base_folder}/enriched_mds/ADAmantium.md"
procesar_flujo_modular(informe_md)

📄 Procesando: /content/20252_Informes/Muestra_Informe2/enriched_mds/ADAmantium.md
🤖 Iniciando análisis modular con gpt-5-mini...

🔹 Procesando dimensión: FORMA

🔹 Procesando dimensión: RIGUROSIDAD

🔹 Procesando dimensión: CONSISTENCIA

🔹 Generando Evaluación Final y Resumen...

VISTA PREVIA DEL FEEDBACK:
# Informe de Retroalimentación Automática

# Informe de Calidad — Algoritmo D.Palindromes

## Análisis de Forma y Estructura

1) Definición de Entradas/Salidas
- Evidencia Encontrada:
  - Sección "Entrada": "Una cadena $s$ que consiste únicamente en letras ... Un entero $k$ que cumple $1 \leq k \leq |s|$ ..."
  - Sección "Salida": "Sea $P$ un numero entero que representa el número mínimo de cambios requeridos"
  - Ejemplo de entrada en la misma sección: "$s =$ `“abdsjkj”`, $k = 4`"
- Análisis:
  - Existe una descripción semántica clara de las entradas: cadena s y entero k con restricciones sobre tamaño y sensibilidad a mayúsculas/minúsculas. La salida también está definida en términos 

##Cambios

-Se descarto GEval y se usa Gemini como evaluador de la respuesta de los feedbacks.

-Se elimino validacion de "Human-in-the-loop" para dar un archivo directamente para su revision

-"CAMBIO IMPORTANTE" Se implementa GPT-5 dando respuestas certeras y la validacion desde Gemini de estas mismas

-Se acorta la evaluacion usando como criterio
 No cumple|Cumple parcialmente|Bien hecho

-